In [6]:
import math
import random
trainingSize=0
inputSize=0
hiddenSize=0
outputSize=0
inputIndex=0

trainingset= None #traning matrix
expectedOutput= None #Class 0 or 1
Wih= None #hidden layer matrix
Who=None #hidden output layer
#hiddenLayerValues=None

Xh=None
Xo=None

learningRate=0.2

testThreshold = 1.0/6.0
trainingThreshold = testThreshold/6

def sigmoid(x):
    return 1/(1+math.exp(-x))

def sigmoidDerivative(x):
    return x*(1-x)

def initWeights():
    global Wih, Who
    Wih=[[0]*hiddenSize for i in range(inputSize)]#input to hidden
    Who=[[0]*outputSize for h in range(hiddenSize)]
    for i in range(inputSize):
        for h in range(hiddenSize):
            Wih[i][h]=random.random()
    for h in range(hiddenSize):
        for o in range(outputSize):
            Who[h][o]=random.random()


def feedForward():
    global Xh,Xo
    Xi=trainingset[inputIndex]
    for h in range(hiddenSize):
       dot=0
       for i in range(inputSize):
          dot+=Xi[i]*Wih[i][h]
       Xh[h]=sigmoid(dot)
       
    for o in range (outputSize):
        dot=0
        for h in range(hiddenSize):
            dot+=Xh[h]*Who[h][o]
        Xo[o]=sigmoid(dot)
        
def backPropagation(inputIndex):
    global Xh, Xo, Wih, Who
    DeltaO=[0]*outputSize
    DeltaHidden=[0]*hiddenSize
    
    for o in range(outputSize):
        DeltaO[o]=(expectedOutput[inputIndex][o]-Xo[o])*sigmoidDerivative(Xo[o])
    
    for o in range(outputSize):
        for h in range(hiddenSize):
            Who[h][o]+=learningRate*DeltaO[o]*Xh[h]
    
    for h in range(hiddenSize):
        dot=0
        for o in range(outputSize):
            dot+=DeltaO[o]*Who[h][o]
        DeltaHidden[h]=sigmoidDerivative(Xh[h])*dot
    
    for i in range(inputSize):
        for h in range(hiddenSize):
            Wih[i][h]+=learningRate*DeltaHidden[h]*trainingset[inputIndex][i]
                    
def isCorrect(index):
    for o in range(outputSize):
        diff= abs(expectedOutput[index][o]-Xo[o])
        if diff > trainingThreshold:
            return False
    return True    
                    

def trainingNeuralNetwork(nEpochs):
    global trainingSize
    initWeights()
    
    hits = 0
    
    for e in range(nEpochs):
        for i in range(inputSize):
            feedForward(i)
            if isCorrect(i):
                hits+=1
                if hits>=trainingSize:
                    return True
            else:
                    hits=0
                    backPropagation(i)

def readTrainingSet(filename):
    global trainingSize, inputSize, outputSize, trainingset, expectedOutput, Xh, Xo
    with open(filename) as f:
        trainingSize, inputSize, outputSize = [int(x) for x in next(f).split()]
        hiddenSize = (inputSize//2)
        Xh=[0]*hiddenSize
        Xo=[0]*outputSize
        trainingSet=[None]*trainingSize
        expectedOutput=[None]*trainingSize
        
        array=[[float(x) for x in line.split()] for line in f if len(line)>1]
        data = [v[x] for v in array for x in range(len(v))]
        
        total= inputSize+outputSize
        for n in range(trainingSize):
            p = n*total
            trainingSet[n]=data[p:p+inputSize]
            expectedOutput[n]=data[p+inputSize:p+total]
    f.close()

def testNN(filename):
    correct=0
    with open(filename) as f:
        trainingSize, inputSize, outputSize = [int(x) for x in next(f).split()]
        array=[[float(x) for x in line.split()] for line in f if len(line)>1]
        data=[v[x] for v in array for x in range(len(v))]   
        total= inputSize+outputSize
        for n in range(trainingSize):
            p = n*total
            trainingset=data[p:p+inputSize]
            expectedOutput=data[p+inputSize:p+total]
            feedForward()
            if isCorrect(0):
                correct+=1
                print("Yes")
            else:
                print("No")
    f.close()
    print("Correct: ", correct, " out of ", trainingSize)
    
def testPattern(pattern,eOut):
    global Wih,Who
    print(pattern,eOut)
    Xi= pattern
    for h in range(hiddenSize):
        dot=0
        for i in range(inputSize):
            dot+= Xi[i]*Wih[i][h]
            Xh[h]=sigmoid(dot)
    for o in range(outputSize):
        dot=0
        for h in range(hiddenSize):
            dot+=Xh[h]*Who[h][o]
            Xo[o]=sigmoid(dot)
    
    for o in range(outputSize):
        diff= abs(eOut[o]-Xo[o])
        if diff >= testThreshold:
            print("No")
    return True                            
                   
            
        
readTrainingSet("training.txt")
